### Imports

In [2]:
# IMPORTS
from distutils.command.config import config
import pickle5 as pickle

# Libraries
import matplotlib.pyplot as plt
import pandas as pd
import wandb
import os
from os.path import exists

# Evaluation
from sklearn.metrics import f1_score, multilabel_confusion_matrix, accuracy_score, classification_report
import seaborn as sns

# Tokenization
from tokenizers import  Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace
import glob

# data
import numpy as np
import matplotlib.pyplot as plt

#torch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader

In [6]:
PKL_PATH = '/home/svetlana.maslenkova/LSTM/pickles/'

with open(PKL_PATH + 'pid_hid_dignoses.pkl', 'rb') as handle:
    data_diagnoses = pickle.load(handle)

data_diagnoses.head()

FileNotFoundError: [Errno 2] No such file or directory: '/home/svetlana.maslenkova/LSTM/pickles/pid_hid_dignoses.pkl'

In [7]:
import pandas as pd

DATA_PATH = '/home/svetlana.maslenkova/data/'

data_admissions = pd.read_csv(DATA_PATH+'mimic-iv-1.0/core/admissions.csv')

data_admissions.columns = data_admissions.columns.str.lower()
data_admissions['hadm_id'] = data_admissions['hadm_id'].astype(int)
format_ = '%Y-%m-%d %H:%M:%S'
data_admissions['admittime'] = pd.to_datetime(data_admissions['admittime'], format=format_)

data_admissions.head(2)

,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,language,marital_status,ethnicity,edregtime,edouttime,hospital_expire_flag
0,14679932,21038362,2139-09-26 14:16:00,2139-09-28 11:30:00,NaN,ELECTIVE,NaN,HOME,Other,ENGLISH,SINGLE,UNKNOWN,NaN,NaN,0
1,15585972,24941086,2123-10-07 23:56:00,2123-10-12 11:22:00,NaN,ELECTIVE,NaN,HOME,Other,ENGLISH,NaN,WHITE,NaN,NaN,0


In [8]:
import pandas as pd

data_diagnoses = pd.read_csv(DATA_PATH+'mimic-iv-1.0/hosp/diagnoses_icd.csv')

data_diagnoses.columns = data_diagnoses.columns.str.lower()

data_diagnoses = data_diagnoses.merge(data_admissions[['hadm_id', 'admittime']])

data_diagnoses.head()

,subject_id,hadm_id,seq_num,icd_code,icd_version,admittime
0,15734973,20475282,3,2825,9,2112-02-02 07:53:00
1,15734973,20475282,2,V0251,9,2112-02-02 07:53:00
2,15734973,20475282,5,V270,9,2112-02-02 07:53:00
3,15734973,20475282,1,64891,9,2112-02-02 07:53:00
4,15734973,20475282,4,66481,9,2112-02-02 07:53:00


## Get diagnoses

In [179]:
# res = pid_train_df.merge(data_admissions[['hadm_id', 'admittime']])
# res = pid_val_df.merge(data_admissions[['hadm_id', 'admittime']])
res = pid_test_df.merge(data_admissions[['hadm_id', 'admittime']])
res.head()

,subject_id,hadm_id,demographics_in_visit,lab_tests_in_visit,medications_in_visit,vitals_in_visit,days_in_visit,aki_status_in_visit,days,admittime
0,17223646,20023461,"[WHITE F 69.0, WHITE F 69.0, WHITE F 69.0, WHI...",[Hematology Blood hematocrit {37.7} %; Hematol...,[Sodium Chloride 0.9% Flush {3} mL ; Heparin ...,[temp {97.7} heartrate {68.0} resprate {18.0} ...,[WHITE F 69.0$temp {97.7} heartrate {68.0} res...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, nan...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",2152-10-31 20:11:00
1,18371764,20046098,"[WHITE M 56.0, WHITE M 56.0, WHITE M 56.0, WHI...",[Hematology Blood hematocrit {40.1} %; Hematol...,[Sodium Chloride 0.9% Flush {3} mL ; D5 1/2NS...,[temp {100.3} heartrate {103.0} resprate {18.0...,[WHITE M 56.0$temp {100.3} heartrate {103.0} r...,"[0, 0, nan, nan]","[0, 1, 2, 3]",2162-09-15 13:43:00
2,17416292,20051972,"[UNKNOWN F 56.0, UNKNOWN F 56.0, UNKNOWN F 56....",[Hematology Blood hematocrit {27.5} %; Hematol...,[ARIPiprazole {10} mg ; Docusate Sodium {100} ...,[temp {} heartrate {68.0} resprate {22.0} o2sa...,[UNKNOWN F 56.0$temp {} heartrate {68.0} respr...,"[0, 0, 0, 1, 1, 1, 1, 1, 1]","[0, 1, 2, 3, 4, 5, 6, 7, 8]",2168-04-06 03:17:00
3,15109929,20058401,"[WHITE M 77.0, WHITE M 77.0, WHITE M 77.0, WHI...",[Hematology Blood hematocrit {24.0} %; Hematol...,"[nan, Sodium Chloride 0.9% Flush {3} mL ; Ace...",[temp {97.9} heartrate {64.0} resprate {18.0} ...,[WHITE M 77.0$temp {97.9} heartrate {64.0} res...,"[0, 0, 0, 0, 0, nan, nan]","[-1, 0, 1, 2, 3, 4, 5]",2157-11-11 01:29:00
4,14687005,20064641,"[HISPANIC/LATINO M 48.0, HISPANIC/LATINO M 48....",[Hematology Blood hematocrit {27.2} %; Hematol...,"[nan, nan, nan, Influenza Vaccine Quadrivalent...",[temp {98.8} heartrate {91.0} resprate {18.0} ...,[HISPANIC/LATINO M 48.0$temp {98.8} heartrate ...,"[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, nan...","[-3, -2, -1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10,...",2159-04-05 10:13:00


In [ ]:
patient = 13131863
admittime = res[res.subject_id==patient].admittime.values[0]
data_diagnoses[(data_diagnoses.subject_id==patient) & (data_diagnoses.admittime < admittime)].sort_values('admittime', ascending=False).icd_code.tolist()[:30]

In [180]:
# patient_previous_diag_dict = {}
i=0
print(f'Total {len(res.subject_id.unique())} patients')
for patient in res.subject_id.unique():
    if i%100==0:
        print(f'{i} patients')
    admittime = res[res.subject_id==patient].admittime.values[0]
    prev_diags = data_diagnoses[(data_diagnoses.subject_id==patient) & (data_diagnoses.admittime < admittime) ].sort_values('admittime', ascending=False).icd_code.tolist()[:30]
    patient_previous_diag_dict[patient] = prev_diags
    i += 1
    


Total 827 patients
0 patients
100 patients
200 patients
300 patients
400 patients
500 patients
600 patients
700 patients
800 patients


In [181]:
len(patient_previous_diag_dict)

10504

In [183]:
df_prev_diagnoses = pd.DataFrame.from_dict(patient_previous_diag_dict, orient='index').reset_index().rename(columns={'index':'hadm_id'}).fillna('PAD')

In [184]:
df_prev_diagnoses['prev_diags'] = df_prev_diagnoses.drop(labels=['hadm_id'], axis=1).apply(' '.join, axis=1)

In [186]:
df_prev_diagnoses = df_prev_diagnoses[['hadm_id', 'prev_diags']]
df_prev_diagnoses

,hadm_id,prev_diags
0,16679562,F17210 E785 J111 K219 F1020 C61 N189 J441 N400...
1,10189736,K921 I7103 J9691 J90 I959 D62 Z87891 K269 D696...
2,19669999,42833 V4581 4280 73313 41400 2724 5770 57450 4...
3,12377638,PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PA...
4,18423382,3051 V1581 42789 44422 99674 6826 4019 44022 V...
...,...,...
10499,18203312,PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PA...
10500,16688191,PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PA...
10501,14644623,PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PA...
10502,18011979,PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PA...


## Classes

### Dataset

In [35]:
class MyDataset(Dataset):

    def __init__(self, df, tokenizer, max_length=400, pred_window=2, observing_window=3):
        self.df = df
        self.tokenizer = tokenizer
        self.observing_window = observing_window
        self.pred_window = pred_window
        self.max_length = max_length

        
    def __len__(self):
        return self.df.shape[0]

    def __getitem__(self, idx):

        return self.make_matrices(idx)
    
    def tokenize(self, text, max_length):
        
        output = self.tokenizer.encode(text)
        # padding and truncation
        if len(output.ids) < max_length:
            len_missing_token = max_length - len(output.ids)
            padding_vec = [self.tokenizer.token_to_id('[PAD]') for _ in range(len_missing_token)]
            token_output = [*output.ids, *padding_vec]
        elif len(output.ids) > max_length:
            token_output = output.ids[:max_length]
        else:
            token_output = output.ids
        
        return token_output

    def make_matrices(self, idx):
        day_info = self.df.days_in_visit.values[idx]
        aki_status = self.df.aki_status_in_visit.values[idx]
        days = self.df.days.values[idx]
        # print(idx)

        aki_happened = False
        labels = []
        day_info_list = []
        label = None

        for day in range(days[0], days[0]+ self.observing_window):
            if day not in days:
                labels.append(0)
                day_info_list.append(self.tokenize('', self.max_length))
            else:
                i = days.index(day)
                
                if (day + self.pred_window) not in days:
                    labels.append(0)
                else:              
                    if ((i + self.pred_window) >= len(aki_status)) or np.isnan(aki_status[i + self.pred_window]):
                        labels.append(0)
                    else:
                        labels.append(aki_status[i + self.pred_window])

                if str(day_info[i]) == 'nan':
                    day_info_list.append(self.tokenize('[PAD]', self.max_length))
                else:
                    day_info_list.append(self.tokenize(day_info[i], self.max_length))

        if sum(labels) > 0:
            label = 1
        else:
            label = 0

        #make tensors
        tensor_day = torch.tensor(day_info_list, dtype=torch.int64)
        tensor_labels = torch.tensor(label, dtype=torch.float64)

        return tensor_day, tensor_labels


class EHR_MODEL(nn.Module):
    def __init__(self, max_length, vocab_size, device, pred_window=2, observing_window=3,  H=128, embedding_size=200):
        super(EHR_MODEL, self).__init__()

        self.observing_window = observing_window
        self.pred_window = pred_window
        self.H = H
        self.max_length = max_length
        self.embedding_size = embedding_size
        self.vocab_size = vocab_size
        self.device = device

        # self.embedding = pretrained_model
        self.embedding = nn.Embedding(self.vocab_size, self.embedding_size)

        self.lstm_day = nn.LSTM(input_size=embedding_size,
                            hidden_size=self.H,
                            num_layers=1,
                            batch_first=True,
                            bidirectional=True)

        self.fc_day = nn.Linear(self.max_length * 2 * self.H, 2048)

        # self.fc_1 = nn.Linear(self.observing_window * 2 * self.H, 2048)

        self.lstm_adm = nn.LSTM(input_size=2048*self.observing_window,
                            hidden_size=self.H,
                            num_layers=2,
                            batch_first=True,
                            bidirectional=True)

        self.drop = nn.Dropout(p=0.5)

        self.fc_2 = nn.Linear(self.H*2, 1)

    def forward(self, tensor_day):

        batch_size = tensor_day.size()[0]

        full_output = torch.tensor([]).to(device=self.device)

        for d in range(self.observing_window):
            # embedding layer applied to all tensors [16,400,200]
            out_emb = self.embedding(tensor_day[:, d, :].squeeze(1))
            # print('out_emb', out_emb.size())

            # lstm layer applied to embedded tensors
            output_lstm_day= self.fc_day(\
                                    self.lstm_day(out_emb)[0]\
                                        .reshape(batch_size, self.max_length * 2 * self.H))

            # print('output_lstm_day', output_lstm_day.size())                   
            # concatenate for all * days
            full_output = torch.cat([full_output, output_lstm_day], dim=1) # [16, 768]

        # print('full_output size: ', full_output.size(), '\n')
        # output = self.fc_1(full_output)
        output, _ = self.lstm_adm(full_output)
        # print('output after lstm_adm', output.size())
        output = self.drop(output)
        output = self.fc_2(output)
        # print('output after fc_2', output.size())
        output = torch.squeeze(output, 1)

        # output = nn.Sigmoid()(output)

        return output

In [64]:
train_dataset = MyDataset(pid_train_df, tokenizer=tokenizer)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

In [65]:
tensor_day, tensor_labels, idx = next(iter(train_loader))

In [68]:
tensor_day.size()

torch.Size([16, 2, 400])

In [66]:
tensor_day

tensor([[[ 265,   44, 1122,  ...,  128,   27,  254],
         [ 265,   44, 1122,  ...,  602,   88,  344]],

        [[ 265,   37, 1151,  ...,    0,    0,    0],
         [ 265,   37, 1151,  ...,    0,    0,    0]],

        [[ 265,   44,  441,  ...,    0,    0,    0],
         [ 265,   44,  441,  ...,    0,    0,    0]],

        ...,

        [[ 265,   37,  441,  ...,    0,    0,    0],
         [ 265,   37,  441,  ...,    0,    0,    0]],

        [[ 265,   44,  907,  ...,   99,  170,   88],
         [ 265,   44,  907,  ...,   15,   43,   27]],

        [[ 265,   37, 1257,  ...,    0,    0,    0],
         [ 265,   37, 1257,  ...,    0,    0,    0]]])

In [67]:
tensor_labels

tensor([0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0.],
       dtype=torch.float64)

In [57]:
idx

tensor([9459, 9040, 1514, 7487, 2421, 7426, 6304, 8595, 4970, 5357, 9396, 6063,
        4573, 2308, 2358, 8064])

In [61]:
pid_train_df.iloc[1514]

subject_id                                                        10921932
hadm_id                                                           21504239
demographics_in_visit    [WHITE M 30.0, WHITE M 30.0, WHITE M 30.0, WHI...
lab_tests_in_visit       [Hematology Blood hematocrit {36.4} %; Hematol...
medications_in_visit     [0.9% Sodium Chloride {250} mL ; D5 1/2NS {100...
vitals_in_visit                        [nan, nan, nan, nan, nan, nan, nan]
days_in_visit            [WHITE M 30.0$nan$Hematology Blood hematocrit ...
aki_status_in_visit                              [0, 0, 1, 0, 0, nan, nan]
days                                                 [0, 1, 2, 3, 4, 5, 6]
Name: 5140, dtype: object

### Model

In [32]:
class EHR_MODEL(nn.Module):
    def __init__(self, max_length, vocab_size, device, pred_window=2, observing_window=3,  H=128, embedding_size=200):
        super(EHR_MODEL, self).__init__()

        self.observing_window = observing_window
        self.pred_window = pred_window
        self.H = H
        self.max_length = max_length
        self.embedding_size = embedding_size
        self.vocab_size = vocab_size
        self.device = device

        # self.embedding = pretrained_model
        self.embedding = nn.Embedding(self.vocab_size, self.embedding_size)

        self.lstm_day = nn.LSTM(input_size=embedding_size,
                            hidden_size=self.H,
                            num_layers=1,
                            batch_first=True,
                            bidirectional=True)

        self.fc_day = nn.Linear(self.max_length * 2 * self.H, 2048)

        # self.fc_1 = nn.Linear(self.observing_window * 2 * self.H, 2048)

        self.lstm_adm = nn.LSTM(input_size=2048*self.observing_window,
                            hidden_size=self.H,
                            num_layers=2,
                            batch_first=True,
                            bidirectional=True)

        self.drop = nn.Dropout(p=0.5)

        self.fc_2 = nn.Linear(self.H*2, 1)

    def forward(self, tensor_day):

        batch_size = tensor_day.size()[0]

        full_output = torch.tensor([]).to(device=self.device)

        for d in range(self.observing_window):
            # embedding layer applied to all tensors [16,400,200]
            out_emb = self.embedding(tensor_day[:, d, :].squeeze(1))
            # print('out_emb', out_emb.size())

            # lstm layer applied to embedded tensors
            output_lstm_day= self.fc_day(\
                                    self.lstm_day(out_emb)[0]\
                                        .reshape(batch_size, self.max_length * 2 * self.H))

            # print('output_lstm_day', output_lstm_day.size())                   
            # concatenate for all * days
            full_output = torch.cat([full_output, output_lstm_day], dim=1) # [16, 768]

        # print('full_output size: ', full_output.size(), '\n')
        # output = self.fc_1(full_output)
        output, _ = self.lstm_adm(full_output)
        # print('output after lstm_adm', output.size())
        output = self.drop(output)
        output = self.fc_2(output)
        # print('output after fc_2', output.size())
        output = torch.squeeze(output, 1)

        # output = nn.Sigmoid()(output)

        return output

In [398]:
device = 'cuda'

In [407]:
model = EHR_MODEL(400,vocab_size, device).to(device)

In [38]:
tensor_day, tensor_labels = next(iter(train_loader))
tensor_day = tensor_day.to(device)
tensor_labels = tensor_labels.to(device)
output = model(tensor_day)


In [409]:
output

tensor([0.5238, 0.5113, 0.5122, 0.5097, 0.5113, 0.5141, 0.5060, 0.5147],
       device='cuda:0', grad_fn=<SigmoidBackward0>)

In [14]:
CURR_PATH = '/home/svetlana.maslenkova/LSTM'
PKL_PATH = CURR_PATH+'/pickles/'
DF_PATH = CURR_PATH +'/dataframes/'

# loading the data
with open(DF_PATH + 'pid_train_df_finetuning.pkl', 'rb') as f:
    pid_train_df = pickle.load(f)

with open(DF_PATH + 'pid_val_df_finetuning.pkl', 'rb') as f:
    pid_val_df = pickle.load(f)

with open(DF_PATH + 'pid_test_df_finetuning.pkl', 'rb') as f:
    pid_test_df = pickle.load(f)

tokenizer = Tokenizer.from_file(CURR_PATH + '/tokenizer.json')

# Training

## Functions

### Utils

In [15]:
# Save and Load Functions
def save_checkpoint(save_path, model, optimizer, valid_loss):

    if save_path == None:
        return
    
    state_dict = {'model_state_dict': model.state_dict(),
                  'optimizer_state_dict': optimizer.state_dict(),
                  'valid_loss': valid_loss}
    
    torch.save(state_dict, save_path)
    print(f'Model saved to ==> {save_path}')


def load_checkpoint(load_path, model, optimizer, device):

    if load_path==None:
        return
    
    state_dict = torch.load(load_path, map_location=device)
    print(f'Model loaded from <== {load_path}')
    
    model.load_state_dict(state_dict['model_state_dict'])
    optimizer.load_state_dict(state_dict['optimizer_state_dict'])
    
    return state_dict['valid_loss']

def load_checkpoint(load_path, model, optimizer, device):

    if load_path==None:
        return
    
    state_dict = torch.load(load_path, map_location=device)
    print(f'Model loaded from <== {load_path}')
    
    model.load_state_dict(state_dict['model_state_dict'])
    optimizer.load_state_dict(state_dict['optimizer_state_dict'])
    
    return state_dict['valid_loss']

In [16]:
def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

### Evaluation function

In [58]:
def evaluate(model, test_loader, device, threshold=0.5, log_res=True):
    model = model.to(device)
    stacked_labels = torch.tensor([]).to(device)
    stacked_preds = torch.tensor([]).to(device)
    
    model.eval()
    step = 1
    with torch.no_grad():
        for tensor_day, tensor_labels in test_loader:
            # print(f'Step {step}/{len(test_loader)}' )
            labels = tensor_labels.to(device)
            day_info = tensor_day.to(device)

            output = model(day_info)
            output = nn.Sigmoid()(output)
            output = (output > threshold).int()

            # stacking labels and predictions
            stacked_labels = torch.cat([stacked_labels, labels], dim=0, )
            stacked_preds = torch.cat([stacked_preds, output], dim=0, )

            step += 1

    # calculate accuracy
    acc = torch.round(torch.sum(stacked_labels==stacked_preds) / len(stacked_labels), decimals=2)
    # transfer to device
    stacked_labels = stacked_labels.cpu().detach().numpy()
    stacked_preds = stacked_preds.cpu().detach().numpy()
    # get classification metrics for all samples in the test set
    classification_report_res = classification_report(stacked_labels, stacked_preds, zero_division=0, output_dict=True)
    print(classification_report(stacked_labels, stacked_preds, zero_division=0, output_dict=False))
    if log_res:
        for k, v in classification_report_res.items():
            if k == 'accuracy':
                 wandb.log({'test_accuracy': v})
            else:
                for k1, v1 in v.items():
                    if k1 == 'support':continue
                    else:
                        wandb.log({'test_'+str(k) +'_'+str(k1): v1})

    return classification_report_res, acc

In [36]:
test_dataset = MyDataset(pid_test_df.sample(frac=0.1), tokenizer=tokenizer, max_length=400)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=True)

In [37]:
vocab_size = tokenizer.get_vocab_size()

In [38]:
model = EHR_MODEL(max_length=400, vocab_size=vocab_size, device='cpu')

In [41]:
classification_report_res, acc = evaluate(model, test_loader, device='cpu', log_res=False)

              precision    recall  f1-score   support

         0.0       0.48      1.00      0.65        83
         1.0       0.00      0.00      0.00        89

    accuracy                           0.48       172
   macro avg       0.24      0.50      0.33       172
weighted avg       0.23      0.48      0.31       172



In [57]:
for k, v in classification_report_res.items():
    if k == 'accuracy':
        print('test_accuracy', v)
    else:
        for k1, v1 in v.items():
            if k1 == 'support':continue
            else:
                print('test_'+str(k) +'_'+str(k1), v1)

test_0.0_precision 0.48255813953488375
test_0.0_recall 1.0
test_0.0_f1-score 0.6509803921568628
test_1.0_precision 0.0
test_1.0_recall 0.0
test_1.0_f1-score 0.0
test_accuracy 0.48255813953488375
test_macro avg_precision 0.24127906976744187
test_macro avg_recall 0.5
test_macro avg_f1-score 0.3254901960784314
test_weighted avg_precision 0.23286235803136832
test_weighted avg_recall 0.48255813953488375
test_weighted avg_f1-score 0.31413588691290467


In [54]:
classification_report_res

{'0.0': {'precision': 0.48255813953488375,
  'recall': 1.0,
  'f1-score': 0.6509803921568628,
  'support': 83},
 '1.0': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 89},
 'accuracy': 0.48255813953488375,
 'macro avg': {'precision': 0.24127906976744187,
  'recall': 0.5,
  'f1-score': 0.3254901960784314,
  'support': 172},
 'weighted avg': {'precision': 0.23286235803136832,
  'recall': 0.48255813953488375,
  'f1-score': 0.31413588691290467,
  'support': 172}}

### Training function

In [60]:
def train(model, 
          optimizer,
          train_loader,
          valid_loader,
          file_path,
          device='cpu',
          num_epochs=5,
          criterion = nn.BCELoss(),
          pos_weight = torch.tensor([]),
          best_valid_loss = float("Inf"),
          dimension=128,
          epoch_patience=15,
          threshold=0.5,
          scheduler=None):
    
    # initialize running values
    running_loss = 0.0
    running_acc = 0.0
    valid_running_loss = 0.0
    valid_running_acc = 0.0
    global_step = 0
    train_loss_list = []
    valid_loss_list = []
    train_acc_list = []
    valid_acc_list = []
    global_steps_list = []
    stop_training = 0

    sigmoid_fn = nn.Sigmoid()

    if criterion == 'BCEWithLogitsLoss':
        criterion = nn.BCEWithLogitsLoss()
        criterion.pos_weight = pos_weight.to(device)
        use_sigmoid=False
    else:
        criterion = nn.BCELoss()
        use_sigmoid = True

    # training loop
    for epoch in range(num_epochs):  
        stacked_labels = torch.tensor([]).to(device)
        stacked_preds = torch.tensor([]).to(device)

        model.train()
        for tensor_day, tensor_labels in train_loader:
            # transferring everything to GPU
            labels = tensor_labels.to(device)
            tensor_day = tensor_day.to(device)

            output = model(tensor_day)

            if use_sigmoid:
                loss = criterion(sigmoid_fn(output), labels.type(torch.float32))
            else:
                loss = criterion(output, labels.type(torch.float32))

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            output_pred = sigmoid_fn(output)
            output_pred = (output_pred > threshold).int()

            stacked_labels = torch.cat([stacked_labels, labels], dim=0)
            stacked_preds = torch.cat([stacked_preds, output_pred], dim=0)
            
            global_step += 1

            wandb.log({'step_train_loss': loss.item(), 'global_step': global_step})
            
        # calculate accuracy
        epoch_train_accuracy = torch.round(torch.sum(stacked_labels==stacked_preds) / len(stacked_labels), decimals=2)
        if scheduler is not None:
            scheduler.step()
            print(f'Learning rate is {get_lr(optimizer)}')

        model.eval()
        stacked_labels = torch.tensor([]).to(device)
        stacked_preds = torch.tensor([]).to(device)
        with torch.no_grad():
            # validation loop
            for tensor_day, tensor_labels in valid_loader:
                labels = tensor_labels.to(device)
                tensor_day = tensor_day.to(device)
                
                output = model(tensor_day)

                if use_sigmoid:
                    loss = criterion(sigmoid_fn(output), labels.type(torch.float32))
                else:
                    loss = criterion(output, labels.type(torch.float32))

                valid_running_loss += loss.item()

                output_pred = sigmoid_fn(output)
                output_pred = (output_pred > threshold).int()

                # stacking labels and predictions
                stacked_labels = torch.cat([stacked_labels, labels], dim=0)
                stacked_preds = torch.cat([stacked_preds, output_pred], dim=0)

        # transfer to device
        stacked_labels = stacked_labels.cpu().detach().numpy()
        stacked_preds = stacked_preds.cpu().detach().numpy()
        # calculate accuracy
        epoch_val_accuracy = np.round(np.sum(stacked_labels==stacked_preds) / len(stacked_labels), 2)
        # get classification metrics for all samples in the test set
        classification_report_res = classification_report(stacked_labels, stacked_preds, zero_division=0, output_dict=True)
        classification_report_res.update({'epoch':epoch+1})

        # log the evaluation metrics 
        for key, value in classification_report_res.items():
            wandb.log({key:value, 'epoch':epoch+1})

        # valid loss
        epoch_average_train_loss = running_loss / len(train_loader)  
        epoch_average_valid_loss = valid_running_loss / len(valid_loader)

        train_loss_list.append(epoch_average_train_loss)
        valid_loss_list.append(epoch_average_valid_loss)
        train_acc_list.append(epoch_train_accuracy)
        valid_acc_list.append(epoch_val_accuracy)


        global_steps_list.append(global_step)
        wandb.log({'epoch_average_train_loss': epoch_average_train_loss,
                    'epoch_average_valid_loss': epoch_average_valid_loss,
                    'epoch_val_accuracy': epoch_val_accuracy, 
                    'epoch_train_accuracy': epoch_train_accuracy,
                    'epoch': epoch+1})

        # resetting running values
        running_loss = 0.0                
        valid_running_loss = 0.0
        
        
        # print progress
        print('Epoch [{}/{}], Step [{}/{}], Train Loss: {:.4f}, Valid Loss: {:.4f}, Valid accuracy: {:.4f}'
                .format(epoch+1, num_epochs, global_step, num_epochs*len(train_loader),
                        epoch_average_train_loss, epoch_average_valid_loss, epoch_val_accuracy))    

        # checkpoint
        if best_valid_loss > epoch_average_valid_loss:
            best_valid_loss = epoch_average_valid_loss
            save_checkpoint(file_path + '/model.pt', model, optimizer, best_valid_loss)
            stop_training = 0
        else:
            stop_training +=1
        
        if stop_training == epoch_patience:
            break


    # save_metrics(file_path + '/metrics.pt', train_loss_list, valid_loss_list, global_steps_list)
    print('Finished Training!')

### Main

In [61]:
def main(saving_folder_name=None, criterion='BCELoss', small_dataset=False,\
     use_gpu=True, project_name='test', pred_window=2, observing_window=3, BATCH_SIZE=128, LR=0.0001,\
         min_frequency=1, hidden_size=128, num_epochs=50, wandb_mode='online', PRETRAINED_PATH=None, run_id=None):
    # define the device
    if use_gpu:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    else:
        device='cpu'

    #paths
    CURR_PATH = os.getcwd()
    PKL_PATH = CURR_PATH+'/pickles/'
    DF_PATH = CURR_PATH +'/dataframes/'
    TXT_DIR_TRAIN = CURR_PATH + '/txt_files/train'
    destination_folder = CURR_PATH + '/training/'


    # Training the tokenizer
    if exists(CURR_PATH + '/tokenizer.json'):
        tokenizer = Tokenizer.from_file(CURR_PATH + '/tokenizer.json')
    else:
        tokenizer = Tokenizer(BPE(unk_token="[UNK]"))
        tokenizer.pre_tokenizer = Whitespace()
        trainer = BpeTrainer(special_tokens=["[PAD]", "[UNK]"], min_frequency=1)
        files = glob.glob(TXT_DIR_TRAIN+'/*')
        tokenizer.train(files, trainer)
        os.environ["TOKENIZERS_PARALLELISM"] = "false"

    # variables for classes
    max_length = 400
    vocab_size = tokenizer.get_vocab_size()
    embedding_size = 200
    dimension = 128
    

    # loading the data
    with open(DF_PATH + 'pid_train_df_finetuning.pkl', 'rb') as f:
        pid_train_df = pickle.load(f)

    with open(DF_PATH + 'pid_val_df_finetuning.pkl', 'rb') as f:
        pid_val_df = pickle.load(f)

    with open(DF_PATH + 'pid_test_df_finetuning.pkl', 'rb') as f:
        pid_test_df = pickle.load(f)

    print('filtering admissions..')
    # filter the admissions
    train_admissions = []
    train_admissions = []
    for adm in pid_train_df.hadm_id.unique():   
        if ({1,2,3,4}.issubset(set(pid_train_df[pid_train_df.hadm_id==adm].days.values[0])) or \
            {-1,0,1,2}.issubset(set(pid_train_df[pid_train_df.hadm_id==adm].days.values[0]))or \
                {0,1,2,3}.issubset(set(pid_train_df[pid_train_df.hadm_id==adm].days.values[0]))) and \
            (len(pid_train_df[pid_train_df.hadm_id==adm].days.values[0])>3) and\
                sum(pid_train_df[pid_train_df.hadm_id==adm].aki_status_in_visit.values[0][:2])==0:
            train_admissions.append(adm)

    val_admissions = []
    for adm in pid_val_df.hadm_id.unique():   
        if ({1,2,3,4}.issubset(set(pid_val_df[pid_val_df.hadm_id==adm].days.values[0])) or \
            {-1,0,1,2}.issubset(set(pid_val_df[pid_val_df.hadm_id==adm].days.values[0]))or \
                {0,1,2,3}.issubset(set(pid_val_df[pid_val_df.hadm_id==adm].days.values[0]))) and \
            (len(pid_val_df[pid_val_df.hadm_id==adm].days.values[0])>3) and\
                sum(pid_val_df[pid_val_df.hadm_id==adm].aki_status_in_visit.values[0][:2])==0:
            val_admissions.append(adm)

    test_admissions = []
    for adm in pid_test_df.hadm_id.unique():   
        if ({1,2,3,4}.issubset(set(pid_test_df[pid_test_df.hadm_id==adm].days.values[0])) or \
            {-1,0,1,2}.issubset(set(pid_test_df[pid_test_df.hadm_id==adm].days.values[0]))or \
                {0,1,2,3}.issubset(set(pid_test_df[pid_test_df.hadm_id==adm].days.values[0]))) and \
            (len(pid_test_df[pid_test_df.hadm_id==adm].days.values[0])>3) and\
                sum(pid_test_df[pid_test_df.hadm_id==adm].aki_status_in_visit.values[0][:2])==0:
            test_admissions.append(adm)

    pid_train_df = pid_train_df[pid_train_df.hadm_id.isin(train_admissions)]
    pid_val_df = pid_val_df[pid_val_df.hadm_id.isin(val_admissions)]
    pid_test_df = pid_test_df[pid_test_df.hadm_id.isin(test_admissions)]

    if small_dataset: frac=0.1
    else: frac=1

    train_dataset = MyDataset(pid_train_df.sample(frac=frac), tokenizer=tokenizer, max_length=400)
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

    val_dataset = MyDataset(pid_val_df.sample(frac=frac), tokenizer=tokenizer, max_length=400)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True)

    test_dataset = MyDataset(pid_test_df.sample(frac=frac), tokenizer=tokenizer, max_length=400)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)


    #print shapes
    tensor_day, tensor_labels = next(iter(train_loader))
    print('\n\n DATA SHAPES: ')
    print('train data shape: ', pid_train_df.shape)
    print('val data shape: ', pid_val_df.shape)
    print('test data shape: ', pid_test_df.shape)

    print('tensor_day', tensor_day.shape)
    print('tensor_labels', tensor_labels.shape)

    # file_path = destination_folder + '/88087_no_weights-lr0.00005-adam'
    
    model = EHR_MODEL(vocab_size=vocab_size, max_length=max_length, device=device, pred_window=2, observing_window=3).to(device)

    if PRETRAINED_PATH is not None:
        model.load_state_dict(torch.load(PRETRAINED_PATH, map_location=device)['model_state_dict'])
        model.embedding.weight.data = model.embedding.weight.data
        print(f"Pretrained model loaded from <=== {PRETRAINED_PATH}")

    optimizer = optim.Adam(model.parameters(), lr=LR)
    # Decay LR by a factor of 0.1 every 7 epochs
    # exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=45, gamma=0.1)
    exp_lr_scheduler = lr_scheduler.MultiStepLR(optimizer, milestones=[15, 50, 80], gamma=0.1)

    train_params = {
                    'model':model,
                    'device':device,
                    'optimizer':optimizer,
                    'criterion':criterion,
                    'train_loader':train_loader,
                    'valid_loader':val_loader,
                    'num_epochs':num_epochs, 
                    'file_path':destination_folder,
                    'best_valid_loss':float("Inf"),
                    'dimension':128,
                    'epoch_patience':10,
                    'threshold':0.5,
                    'scheduler':exp_lr_scheduler
                }

    # path for the model
    if saving_folder_name is None:
        saving_folder_name = 'FX_NO_PRETRAINING_' + str(len(train_loader)*BATCH_SIZE // 1000) + 'k_'  \
            + 'lr' + str(LR) + '_h'+ str(hidden_size) + '_pw' + str(pred_window) + '_ow' + str(observing_window)
    
    file_path = destination_folder + saving_folder_name
    train_params['file_path'] = file_path

    print(f'\n\nMODEL PATH: {file_path}')
    if not os.path.exists(file_path):
        os.makedirs(file_path)
    run_name = saving_folder_name

    # wandb setup
    os.environ['WANDB_API_KEY'] = '8e859a0fc58f296096842a367ca532717d3b4059'
    
    if run_id is None:    
        run_id = wandb.util.generate_id()  
        resume = 'allow' 
    else:
        resume = 'must'
        
    args = {'optimizer':optimizer, 'criterion':'BCELoss', 'LR':LR, 'min_frequency':min_frequency, 'hidden_size':hidden_size, 'pred_window':pred_window, 'experiment':'no_pretraining'}
    wandb.init(project=project_name, name=run_name, mode=wandb_mode, config=args, id=run_id, resume=resume)
    print('Run id is: ', run_id)

    # training
    print('Training started..')
    train(**train_params)

    # testing
    print('\nTesting the model...')
    load_checkpoint(file_path + '/model.pt', model, optimizer, device=device)
    evaluate(model, test_loader, device, threshold=0.5, log_res=True)

    wandb.finish()

### Check the ratio of the data

In [62]:
# loading the data
with open(DF_PATH + 'pid_train_df_finetuning.pkl', 'rb') as f:
    pid_train_df = pickle.load(f)

with open(DF_PATH + 'pid_val_df_finetuning.pkl', 'rb') as f:
    pid_val_df = pickle.load(f)

with open(DF_PATH + 'pid_test_df_finetuning.pkl', 'rb') as f:
    pid_test_df = pickle.load(f)

train_admissions = []
for adm in pid_train_df.hadm_id.unique():   
    if ({1,2,3,4}.issubset(set(pid_train_df[pid_train_df.hadm_id==adm].days.values[0])) or \
        {-1,0,1,2}.issubset(set(pid_train_df[pid_train_df.hadm_id==adm].days.values[0]))or \
            {0,1,2,3}.issubset(set(pid_train_df[pid_train_df.hadm_id==adm].days.values[0]))) and \
        (len(pid_train_df[pid_train_df.hadm_id==adm].days.values[0])>3) and\
            sum(pid_train_df[pid_train_df.hadm_id==adm].aki_status_in_visit.values[0][:2])==0:
        train_admissions.append(adm)

val_admissions = []
for adm in pid_val_df.hadm_id.unique():   
    if ({1,2,3,4}.issubset(set(pid_val_df[pid_val_df.hadm_id==adm].days.values[0])) or \
        {-1,0,1,2}.issubset(set(pid_val_df[pid_val_df.hadm_id==adm].days.values[0]))or \
            {0,1,2,3}.issubset(set(pid_val_df[pid_val_df.hadm_id==adm].days.values[0]))) and \
        (len(pid_val_df[pid_val_df.hadm_id==adm].days.values[0])>3) and\
            sum(pid_val_df[pid_val_df.hadm_id==adm].aki_status_in_visit.values[0][:2])==0:
        val_admissions.append(adm)

test_admissions = []
for adm in pid_test_df.hadm_id.unique():   
    if ({1,2,3,4}.issubset(set(pid_test_df[pid_test_df.hadm_id==adm].days.values[0])) or \
        {-1,0,1,2}.issubset(set(pid_test_df[pid_test_df.hadm_id==adm].days.values[0]))or \
            {0,1,2,3}.issubset(set(pid_test_df[pid_test_df.hadm_id==adm].days.values[0]))) and \
        (len(pid_test_df[pid_test_df.hadm_id==adm].days.values[0])>3) and\
            sum(pid_test_df[pid_test_df.hadm_id==adm].aki_status_in_visit.values[0][:2])==0:
        test_admissions.append(adm)


In [63]:
pid_train_df = pid_train_df[pid_train_df.hadm_id.isin(train_admissions)]
pid_val_df = pid_val_df[pid_val_df.hadm_id.isin(val_admissions)]
pid_test_df = pid_test_df[pid_test_df.hadm_id.isin(test_admissions)]


train_dataset = MyDataset(pid_train_df.sample(frac=1), tokenizer=tokenizer, max_length=400)
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)

val_dataset = MyDataset(pid_val_df.sample(frac=1), tokenizer=tokenizer, max_length=400)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=True)

In [104]:
train_negatives = 0
train_positives = 0
for _, row in pid_train_df.iterrows():
    if (row.aki_status_in_visit[3:5] == [0, 0]) or (row.aki_status_in_visit[3:5] == [np.nan, 0]) or (row.aki_status_in_visit[3:5] == [0, np.nan]):
        train_negatives += 1
    else:
        train_positives += 1

val_negatives = 0
val_positives = 0
for _, row in pid_val_df.iterrows():
    if (row.aki_status_in_visit[3:5] == [0, 0]) or (row.aki_status_in_visit[3:5] == [np.nan, 0]) or (row.aki_status_in_visit[3:5] == [0, np.nan]):
        val_negatives += 1
    else:
        val_positives += 1

test_negatives = 0
test_positives = 0
for _, row in pid_test_df.iterrows():
    if (row.aki_status_in_visit[3:5] == [0, 0]) or (row.aki_status_in_visit[3:5] == [np.nan, 0]) or (row.aki_status_in_visit[3:5] == [0, np.nan]):
        test_negatives += 1
    else:
        test_positives += 1

In [105]:
print(f'Train -  Number of pos: {train_positives}, neg: {train_negatives}, total: {pid_train_df.shape[0]}, ratio of positives: {train_positives/(pid_train_df.shape[0])}')
print(f'Validation -  Number of pos: {val_positives}, neg: {val_negatives}, total: {pid_val_df.shape[0]}, ratio of positives: {val_positives/(pid_val_df.shape[0])}')
print(f'Test -  Number of pos: {test_positives}, neg: {test_negatives}, total: {pid_test_df.shape[0]}, ratio of positives: {test_positives/(pid_test_df.shape[0])}')

Train -  Number of pos: 4962, neg: 5005, total: 9967, ratio of positives: 0.49784288150897965
Validation -  Number of pos: 492, neg: 405, total: 897, ratio of positives: 0.5484949832775919
Test -  Number of pos: 439, neg: 397, total: 836, ratio of positives: 0.5251196172248804


In [35]:
#paths
CURR_PATH = os.getcwd()
PKL_PATH = CURR_PATH+'/pickles/'
DF_PATH = CURR_PATH +'/dataframes/'
TXT_DIR_TRAIN = CURR_PATH + '/txt_files/train'
destination_folder = CURR_PATH + '/training/'

tokenizer = Tokenizer.from_file(CURR_PATH + '/tokenizer.json')
vocab_size = tokenizer.get_vocab_size()

device = 'cuda'

In [36]:
model = EHR_MODEL(vocab_size=vocab_size, max_length=400, device=device, pred_window=2, observing_window=3)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-7, weight_decay=1e-2)

# Run main

In [29]:
PRETRAINED_PATH = '/home/svetlanamaslenkova/Documents/AKI_deep/LSTM/training/FX_NO_PRETRAINING_10k_lr0.001_h128_pw2_ow3/model.pt'
run_id = '3uu5b8vw'

In [30]:
main(saving_folder_name=None, criterion='BCELoss', small_dataset=False,\
     use_gpu=True, project_name='Fixed_obs_window_model', pred_window=2, BATCH_SIZE=800, LR=1e-05,\
         min_frequency=1, hidden_size=128, num_epochs=100, wandb_mode='online', PRETRAINED_PATH=PRETRAINED_PATH, run_id=None)



 DATA SHAPES: 
train data shape:  (9967, 9)
val data shape:  (897, 9)
test data shape:  (836, 9)
tensor_day torch.Size([512, 3, 400])
tensor_labels torch.Size([512])
Pretrained model loaded from <=== /home/svetlanamaslenkova/Documents/AKI_deep/LSTM/training/FX_NO_PRETRAINING_10k_lr0.001_h128_pw2_ow3/model.pt


MODEL PATH: /home/svetlanamaslenkova/Documents/AKI_deep/LSTM/training/FX_NO_PRETRAINING_10k_lr1e-05_h128_pw2_ow3


Run id is:  3uu5b8vw
Training started..
Learning rate is 1e-05
Epoch [1/30], Step [20/600], Train Loss: 0.6470, Valid Loss: 0.6299, Valid accuracy: 0.6500
Model saved to ==> /home/svetlanamaslenkova/Documents/AKI_deep/LSTM/training/FX_NO_PRETRAINING_10k_lr1e-05_h128_pw2_ow3/model.pt
Learning rate is 1e-05
Epoch [2/30], Step [40/600], Train Loss: 0.6458, Valid Loss: 0.6298, Valid accuracy: 0.6400
Model saved to ==> /home/svetlanamaslenkova/Documents/AKI_deep/LSTM/training/FX_NO_PRETRAINING_10k_lr1e-05_h128_pw2_ow3/model.pt
Learning rate is 1e-05
Epoch [3/30], Step [60/600], Train Loss: 0.6453, Valid Loss: 0.6292, Valid accuracy: 0.6500
Model saved to ==> /home/svetlanamaslenkova/Documents/AKI_deep/LSTM/training/FX_NO_PRETRAINING_10k_lr1e-05_h128_pw2_ow3/model.pt
Learning rate is 1e-05
Epoch [4/30], Step [80/600], Train Loss: 0.6449, Valid Loss: 0.6326, Valid accuracy: 0.6400
Learning rate is 1e-05
Epoch [5/30], Step [100/600], Train Loss: 0.6441, Valid Loss: 0.6299, Valid accuracy: 0.65

accuracy,█▆▇▇█▇▇█▆▆▇▇▁
epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▅▆▆▆▇▇▇▇▇▇▇███
epoch_average_train_loss,█▆▄▄▂▆▂▃▁▂▁▃
epoch_average_valid_loss,▅▅▄█▅▃▁▄▂▃▂▅
epoch_train_accuracy,▁▁▁▁▁▁▁▁▁▁▁▁
epoch_val_accuracy,█▁█▁█▁▁█▁▁██
global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
step_train_loss,▆▅▄█▃█▆▅▄▄▅▅▄▄▃▆▅▄▄▄▃▅▆▂▄▃▄▃▅▄▅▁▄▆▃▅▄▄▄▇
accuracy,0.61364
epoch,12
epoch_average_train_loss,0.64451
